# DieselWolf Training Tutorial
This notebook trains a simple AMR classifier using PyTorch Lightning. The goal is to illustrate the training loop and callbacks provided in the repository.

In [ ]:
import make_paths_absolute
from dieselwolf.data import DigitalModulationDataset
from dieselwolf.data.TransformsRF import AWGN
from dieselwolf.models import AMRClassifier, ConfigurableMobileRaT
from dieselwolf.callbacks import SNRCurriculumCallback
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor
from torch.utils.data import DataLoader

In [ ]:
train_ds = DigitalModulationDataset(num_examples=50, num_samples=128, transform=AWGN(20))
val_ds = DigitalModulationDataset(num_examples=50, num_samples=128, transform=AWGN(20))
train_loader = DataLoader(train_ds, batch_size=2, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=2)

In [ ]:
backbone = ConfigurableMobileRaT(seq_len=128, num_classes=len(train_ds.classes))
model = AMRClassifier(backbone, num_classes=len(train_ds.classes), warmup_steps=0)

In [ ]:
callbacks=[LearningRateMonitor(), SNRCurriculumCallback(train_ds, start_snr=20, patience=1)]
trainer = pl.Trainer(max_epochs=1, enable_progress_bar=False)
trainer.fit(model, train_loader, val_loader)

The model trains for a single epoch for speed. In a real experiment you would run for many more epochs.